In [1]:
# Importing the libraries

import cv2
import pickle
import cvzone
import numpy as np

In [2]:
# Width and height of individual parking lot
width, height = 105, 47

# Checking if file CarParkPos is present else initialise posList as an empty list
try:
    with open("CarParkPos", 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []

# function to perform different operations based on the mouse clicks
def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN: # mouseClick - left button(add coordinate points to the posList)
        posList.append((x,y))

    if events == cv2.EVENT_RBUTTONDOWN: # mouseClick - right button(remove the box by poping the points from the posList)
        for i, pos in enumerate(posList):
            x1,y1 = pos
            if x1<x<x1+width and y1<y<y1+height:
                posList.pop(i)

    # Saving the coordinate points in CarParkPos file
    with open("CarParkPos", 'wb') as f:
        pickle.dump(posList, f)

while True:
    # Reading the image
    img = cv2.imread("carParkImg.png")

    # draw the boxes for the individual parking lot
    for pos in posList:
        cv2.rectangle(img, pos, (pos[0]+width, pos[1]+height), (255,0,255), 2)

    # Display the image
    cv2.imshow("Image", img)
    cv2.setMouseCallback("Image", mouseClick)
    cv2.waitKey(1)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ASCII code for 'ESC' key
        break

cv2.destroyAllWindows()

In [3]:
# Capture the video
cap = cv2.VideoCapture("carPark.mp4")

# Load the CarParkPos file
with open("CarParkPos", 'rb') as f:
        posList = pickle.load(f)

width, height = 105, 47


def checkParkingSpace(imgPro):
    spaceCounter = 0
    for pos in posList:
        x,y = pos
        imgCrop = imgPro[y:y+height, x:x+width] # Cropping the image
#         cv2.imshow(str(x*y), imgCrop)
        count = cv2.countNonZero(imgCrop) # Counting the non zero pixels

        if count < 900: # if count<900: parking available else parking occupied
            color =(0, 255, 0)
            thickness = 5
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2

        cv2.rectangle(img, pos, (pos[0]+width, pos[1]+height), color, thickness) # Drawing the rect for each parking lot
        cvzone.putTextRect(img, str(count), (x,y+height-3), scale=1, offset=0, thickness=2, colorR=color) # text to display the number of non zero pixels
    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (100,50), scale=3, offset=20, thickness=5, colorR=(0,200,0)) # text to display free space

while True:
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT): # setting the frames to 0 to loop the video 
        cap.set(cv2.CAP_PROP_POS_FRAMES,0)

    success, img = cap.read()
    
    # image preprocessing
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 1)
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)
    imgMedian = cv2.medianBlur(imgThreshold, 5)
    kernel = np.ones((3,3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

    checkParkingSpace(imgDilate)

    cv2.imshow("Image", img)
#     cv2.imshow("Image Blur", imgBlur)
#     cv2.imshow("Image Thresh", imgThreshold)
#     cv2.imshow("Image Median Blur", imgMedian)
#     cv2.imshow("Image Dilate", imgDilate)
    cv2.waitKey(1)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ASCII code for 'ESC' key
        break

cv2.destroyAllWindows()